In [2]:
import numpy as np
import datasets

datasets.downloads = '/users/gregc/Downloads/'

In [2]:
trX, teX, trY, teY = datasets.mnist()

In [121]:
decode = lambda p: np.argmax(p, axis = 1) 
accuracy = lambda p, Y: np.mean(decode(p) == decode(Y))

In [4]:
import tensorflow as tf

def dense(output_dim, x):
    shape = (x.get_shape()[1].value, output_dim)
    w = tf.Variable(tf.random_normal_initializer(0.0, 0.05)(shape))
    b = tf.Variable(tf.zeros(shape[1], dtype = tf.float32))
    return tf.matmul(x, w) + b  
    
def fit(sess, x, y, X, Y, optimizer, epochs):
    sess.run(tf.initialize_all_variables())
    N, D = X.shape

    for _ in range(epochs):
        for start in range(0, N - 32, 32):
            end = start + 32
            feed_dict = {x: X[start : end], y: Y[start : end]}
            sess.run(optimizer, feed_dict = feed_dict)
            
x = tf.placeholder(tf.float32, (None, trX.shape[1]))
y = tf.placeholder(tf.float32)
h = tf.nn.relu(dense(625, x))  
model = dense(trY.shape[1], h)
loss = tf.nn.softmax_cross_entropy_with_logits(model, y)
optimizer = tf.train.AdamOptimizer().minimize(loss)

with tf.Session() as sess:
    fit(sess, x, y, trX, trY, optimizer, epochs = 1)
    print accuracy(model.eval({x: teX}), teY)

In [73]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(output_dim = 625, input_dim = trX.shape[1], activation = 'relu'))
model.add(Dense(output_dim = trY.shape[1], activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')
model.fit(trX, trY, nb_epoch = 1)
print accuracy(model.predict_proba(teX, verbose = 0), teY)

Epoch 1/1
60000/60000 [==============================] - 32s - loss: 0.1944    
0.9691


In [7]:
config = model.get_config()
weights = model.get_weights()

model = Sequential.from_config(config)
model.set_weights(weights)

In [ ]:
# https://github.com/Newmu/Theano-Tutorials

import theano
from theano import tensor as T

x = T.fmatrix()
y = T.fmatrix()

weights = lambda n, d: theano.shared(np.random.randn(n, d))

w_h = weights(D, H)
w_o = weights(H, C)

l1 = T.nnet.sigmoid(T.dot(x, w_h))
l2 = T.nnet.softmax(T.dot(l1, w_o))

cost = T.mean(T.nnet.categorical_crossentropy(l2, y))
updates = [[p, p - T.grad(cost, p) * learning_rate] for p in [w_h, w_o]]
optimizer = theano.function([x, y], updates = updates, allow_input_downcast = True)

for _ in range(epochs):
    for start, end in zip(range(0, N, 128), range(128, N, 128)):
        optimizer(X[start : end], Y[start : end])
        
w_h, w_o = w_h.get_value(), w_o.get_value()
print accuracy(softmax(expit(X.dot(w_h)).dot(w_o)))

In [184]:
import numpy as np
from softmax import softmax

class Dense:
    def __init__(self, output_dim, X, init_weights = None):
        self.X = X.X
        self.W = self.init_weights(self.X.shape[1], output_dim)
        self.b = np.zeros((output_dim, 1))
        
    @staticmethod
    def init_weights(fan_in, fan_out):
        return np.random.randn(fan_out, fan_in) * 0.05
        
    def forward(self):
        self.X = self.W.dot(self.X.T)
        B = np.tile(self.b, (1, self.X.shape[1]))
        self.X += B

    def back(self):
        pass

    def update(self):
        pass
    
class Relu:
    def __init__(self, X):
        self.X = X
        
    def forward(self):
        self.X = np.maximum(0, self.X.forward())

    def back(self):
        pass
    
class Softmax:
    def __init__(self, X):
        self.X = X
        
    def forward(self):
        self.X = softmax(X)

    def back(self):
        pass

def fit(model, x, y, trX, trY, epochs = 1, batch_size = 2, learning_rate = 0.001, regulizer = None,
       loss = None, optimizer = None):
    for _ in range(epochs):
        for start in range(0, len(trX) - batch_size + 1, batch_size):
            end = start + batch_size
            x.X = trX[start : end]
            y.X = trY[start : end]
            model.forward()

class Placeholder:
    def __init__(self, X):
        self.X = X

trX = np.array([[1, 1], [0, 2], [4, 3], [4, 12]])
trY = np.array([[0, 1], [0, 1], [1, 0], [0, 1]])
x = Placeholder(trX)
y = Placeholder(trY)

model = Softmax(Dense(100, x))
fit(model, x, y, trX, trY)